# Imports

In [1]:
import numpy as np
import pandas as pd 

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Loading Data

In [3]:
train_path = "../input/datathon-2024/train.csv"
test_path = "../input/datathon-2024/test_x.csv"

train_raw = pd.read_csv(train_path)
test_raw = pd.read_csv(test_path)

/tmp/ipykernel_36/2211105482.py:4: DtypeWarning: Columns (13,15,19,30,32,37,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  train_raw = pd.read_csv(train_path)


# Preprocessing

In [4]:
#train_raw.head()
#test_raw.head()

#train_raw.describe()
#test_raw.describe()

train_raw["Cinsiyet"] = train_raw["Cinsiyet"].str.lower()
train_raw["Cinsiyet"].value_counts()

Cinsiyet
erkek                    32764
kadın                    32077
belirtmek istemiyorum      115
Name: count, dtype: int64

## Features

### Doğum tarihi

In [62]:
import re
from datetime import datetime

# TODO! VERİDE DOĞRU OLMAYAN TARİHLER VAR ONLARI TEMİZLEYİP BUNU UYGULA
def tarih_formatla(df):
    def extract_year(date_str):
        if not isinstance(date_str, str):
            return None
        
        # Farklı tarih formatları için regex pattern'leri
        patterns = [
            r'(\d{4})[-/.](\d{1,2})[-/.](\d{1,2})',  # YYYY-MM-DD, YYYY/MM/DD, YYYY.MM.DD
            r'(\d{1,2})[-/.](\d{1,2})[-/.](\d{4})',  # DD-MM-YYYY, DD/MM/YYYY, DD.MM.YYYY
            r'(\d{2})[-/.](\d{1,2})[-/.](\d{1,2})',  # YY-MM-DD, YY/MM/DD, YY.MM.DD
            r'(\d{1,2})[-/.](\d{1,2})[-/.](\d{2})'   # DD-MM-YY, DD/MM/YY, DD.MM.YY
        ]
        
        for pattern in patterns:
            match = re.search(pattern, date_str)
            if match:
                groups = match.groups()
                if len(groups[0]) == 4:  # YYYY-MM-DD formatı
                    year = int(groups[0])
                elif len(groups[2]) == 4:  # DD-MM-YYYY formatı
                    year = int(groups[2])
                else:  # YY-MM-DD veya DD-MM-YY formatı
                    year = int(groups[0] if len(groups[0]) == 2 else groups[2])
                    year += 1900 if year > 24 else 2000
                return year
        
        return None
    
    def yas_sinirla(entry):
        dogum_tarihi = entry['Dogum Tarihi']
        basvurudugu_yil = entry['Basvuru Yili']
        return dogum_tarihi if 1900 <= dogum_tarihi <= basvurudugu_yil - 18 else pd.NA
    
    # Yılları çıkar
    df['Dogum Tarihi'] = df["Dogum Tarihi"].apply(extract_year)
    df['Dogum Tarihi'] = df['Dogum Tarihi'].apply(lambda x: x if 1900 <= x <= 2007 else pd.NA)
    df['yas'] = 2024 - df['Dogum Tarihi']
    # Geçersiz yılları ortalama ile doldur
    mean_year = df['yas'].mean()
    df['yas'] = df['yas'].fillna(round(mean_year)).astype(int)
    
    
    return df

train_raw = tarih_formatla(train_raw)
train_raw["yas"].value_counts()

/tmp/ipykernel_36/3637388605.py:43: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['yas'] = df['yas'].fillna(round(mean_year)).astype(int)


yas
27    18485
24     6341
26     6080
25     5941
23     5089
28     4983
29     4343
30     3283
22     2966
31     2383
54     1713
32     1476
33      846
21      737
34      240
35       62
36       34
37       25
20       24
38       18
19       10
40        6
17        5
39        5
95        4
41        4
99        4
48        3
44        2
47        2
45        2
18        2
97        2
42        1
52        1
91        1
80        1
74        1
Name: count, dtype: int64

In [31]:
train_raw["Dogum Tarihi"].value_counts(dropna=False)

Dogum Tarihi
1/1/70 2:00         1655
1/1/00 0:00          179
NaN                  177
1/1/01 0:00          154
1/1/1997             139
                    ... 
29/08/2000             1
7/30/1991              1
14/05/1996             1
15/12/1997             1
12.11.2003 00:00       1
Name: count, Length: 13499, dtype: int64

In [60]:
train_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65125 entries, 0 to 65124
Data columns (total 45 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   Basvuru Yili                                               65125 non-null  int64  
 1   Degerlendirme Puani                                        65124 non-null  float64
 2   Cinsiyet                                                   64956 non-null  object 
 3   Dogum Tarihi                                               52344 non-null  object 
 4   Dogum Yeri                                                 64334 non-null  object 
 5   Ikametgah Sehri                                            63088 non-null  object 
 6   Universite Adi                                             64993 non-null  object 
 7   Universite Turu                                            64870 non-null  object 
 8   Burslu